In [ ]:
# Importing all necessary libraries 
import tensorflow.keras
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import argparse
import os
import string
import cv2 
import os
import urllib.request
import urllib.parse
import eel
from PIL import Image, ImageOps  
from imutils.video import VideoStream
from imutils.video import FPS
from imutils.object_detection import non_max_suppression
import imutils
import time

eel.init('site')
                                     #Frames Creation
                    
def frame(cust):
    # Read the video from specified path 
    #print(cust)
    s=cust.split('\\')
    ch=s[len(s)-1]
    print(ch)
    cam = cv2.VideoCapture(ch) 
    try: 
        # creating a folder named data 
        if not os.path.exists('data'): 
            os.makedirs('data') 

    # if not created then raise error 
    except OSError: 
        print ('Error: Creating directory of data') 
    # frame 
    currentframe = 0
    while(True): 
        # reading from frame 
        ret,frame = cam.read()
        if ret: 
            # if video is still left continue creating images 
            name = './data/frame' + str(currentframe) + '.jpg'
            #print ('Creating...' + name) 
        # writing the extracted images 
            cv2.imwrite(name, frame) 
# increasing counter so that it will 
# show how many frames are created 
            currentframe += 1
        else: 
            break
# Release all space and windows once done 
    cam.release() 
    cv2.destroyAllWindows()                
        
                                                    #helmate Detection
    
@eel.expose                                             # Expose this function to Javascript
def helmetDetect(cust):
    frame(cust)
# Disable scientific notation for clarity
    np.set_printoptions(suppress=True)
#Load the model
    model = tensorflow.keras.models.load_model('keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
    image = Image.open('data\\frame3.jpg')
    #a=cv2.imread('data\\frame3.jpg')
    #cv2.imshow("OriginalIamge",a)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

# Make sure to resize all images to 224, 224 otherwise they won't fit in the array
    image = image.resize((224, 224))
    image_array = np.asarray(image)

# Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
    data[0] = normalized_image_array
#Run the interface
    prediction = model.predict(data)
    print(prediction)

    result=prediction[0,1]
    if result < 0.05:
        print("Weraed Helmate")
        eel.my_helmet(0)
    else:
        print("Not Weraed Helmate")
        numberplate()
        eel.my_helmet(1)
    

                                              #Triple seat
@eel.expose                                             # Expose this function to Javascript            
def tripleDetect(cust):
    frame(cust)
# Disable scientific notation for clarity
    np.set_printoptions(suppress=True)

# Load the model
    model = tensorflow.keras.models.load_model('keras_modeltriple.h5')


# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


# Replace this with the path to your image
    image = Image.open('data\\frame10.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
    image_array = np.asarray(image)

# display the resized image
    #image.show()

# Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
    data[0] = normalized_image_array

# run the inference
    prediction = model.predict(data)
    result=prediction[0,1]
    print(prediction)
    if result < 0.005:
        print("not Triple Seat")
        eel.my_triple(0)
    else:
        print("Triple Seat")
        numberplate()
        eel.my_triple(1)

                             #number plate Detection
def numberplate():                                       
        image = cv2.imread('data\\frame3.jpg')         #m2.mp4
        #cv2.imshow("original", image)
        #cv2.waitKey(0)
    
    # crop the image using array slices -- it's a NumPy array
        cropped = image[780:900, 760:950]        
        #cv2.imshow("cropped", cropped)
        cv2.imwrite("crop.png", cropped)
        #cv2.waitKey(0)
    
    # load the example image and convert it to grayscale
        image = cv2.imread("crop.jpg")
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # we should apply thresholding to preprocess the image
        preprocess = "thresh"
        gray = cv2.threshold(gray, 0, 255,
                cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
 
     # write the grayscale image to disk as a temporary file so we can apply OCR to it
        filename = "{}.png".format(os.getpid())
        cv2.imwrite(filename, gray)
    
        pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

    # load the image as a PIL/Pillow image, apply OCR, and then delete the temporary file
        text = pytesseract.image_to_string(Image.open(filename))
        os.remove(filename)
    # show the output images
        #cv2.imshow("Image", image)
        #cv2.imshow("Output", gray)
    
        file = open("file.txt", 'r')
        lines = file.readlines()
        file.close()
    
    #remove space and newline from string
        lis=(text.strip()).split()
        st1="".join(lis)
        print("Number Plate "+st1)

        for line in lines:
            parts = line.split() # split line into parts
            if len(parts) > 1:
                column1 = parts[0]
                if column1==st1:
                    column2 = parts[1]
                    print("Mobile Number "+column2)
                
                                                    #SMS Module
        
        def sendSMS(apikey, numbers, sender, message):
            params = {'apikey':apikey , 'numbers': numbers,'sender': sender, 'message' : message}
            f = urllib.request.urlopen('https://api.textlocal.in/send/?'
                + urllib.parse.urlencode(params))
            return (f.read(), f.code)
  
        APIKEY='vS5w1xFYmOU-u2ak0YfXREulHMgkgY9d6B9ea6UuSP'
        resp, code = sendSMS(APIKEY, '123456786','TXTLCL', 'You are voilated the traffic rule ')
        print (resp)
        
                                            #sign Detection
@eel.expose 
def signDetect(cust):
    def decode_predictions(scores, geometry):
    # grab the number of rows and columns from the scores volume, then
    # initialize our set of bounding box rectangles and corresponding
    # confidence scores
        (numRows, numCols) = scores.shape[2:4]
        rects = []
        confidences = []
 
    # loop over the number of rows
        for y in range(0, numRows):
        # extract the scores (probabilities), followed by the
        # geometrical data used to derive potential bounding box
        # coordinates that surround text
            scoresData = scores[0, 0, y]
            xData0 = geometry[0, 0, y]
            xData1 = geometry[0, 1, y]
            xData2 = geometry[0, 2, y]
            xData3 = geometry[0, 3, y]
            anglesData = geometry[0, 4, y]
 
        # loop over the number of columns
            for x in range(0, numCols):
            # if our score does not have sufficient probability,
            # ignore it
                if scoresData[x] < confidence:
                    continue
 
            # compute the offset factor as our resulting feature
            # maps will be 4x smaller than the input image
                (offsetX, offsetY) = (x * 4.0, y * 4.0)
 
            # extract the rotation angle for the prediction and
            # then compute the sin and cosine
                angle = anglesData[x]
                cos = np.cos(angle)
                sin = np.sin(angle)
 
            # use the geometry volume to derive the width and height
            # of the bounding box
                h = xData0[x] + xData2[x]
                w = xData1[x] + xData3[x]
 
            # compute both the starting and ending (x, y)-coordinates
            # for the text prediction bounding box
                endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
                endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
                startX = int(endX - w)
                startY = int(endY - h)
 
            # add the bounding box coordinates and probability score
            # to our respective lists
                rects.append((startX, startY, endX, endY))
                confidences.append(scoresData[x])
 
    # return a tuple of the bounding boxes and associated confidences
        return (rects, confidences)
    east="frozen_east_text_detection.pb"
    #print(cust)
    s=cust.split('\\')
    ch=s[len(s)-1]
    print(ch)
    video=ch
    confidence=0.5
    width=320
    height=320

    (W, H) = (None, None)
    (newW, newH) = (width, height)
    (rW, rH) = (None, None)
 
    # define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
    layerNames = [
        "feature_fusion/Conv_7/Sigmoid",
        "feature_fusion/concat_3"]
 
    # load the pre-trained EAST text detector
    print("[INFO] loading EAST text detector...")
    net = cv2.dnn.readNet(east)
 #  grab a reference to the video file
    vs = cv2.VideoCapture(video)
 
 # start the FPS throughput estimator
    fps = FPS().start()
# loop over frames from the video stream
    while True:
    # grab the current frame, then handle if we are using a
    # VideoStream or VideoCapture object
        frame = vs.read()
        frame = frame[1] 
    # check to see if we have reached the end of the stream
        if frame is None:
            break
 
    # resize the frame, maintaining the aspect ratio
        frame = imutils.resize(frame, width=1000)
        orig = frame.copy()
 
    # if our frame dimensions are None, we still need to compute the
    # ratio of old frame dimensions to new frame dimensions
        if W is None or H is None:
            (H, W) = frame.shape[:2]
            rW = W / float(newW)
            rH = H / float(newH)
 
    # resize the frame, this time ignoring aspect ratio
        frame = cv2.resize(frame, (newW, newH))
# construct a blob from the frame and then perform a forward pass
    # of the model to obtain the two output layer sets
        blob = cv2.dnn.blobFromImage(frame, 1.0, (newW, newH),
            (123.68, 116.78, 103.94), swapRB=True, crop=False)
        net.setInput(blob)
        (scores, geometry) = net.forward(layerNames)
 
    # decode the predictions, then  apply non-maxima suppression to
    # suppress weak, overlapping bounding boxes
        (rects, confidences) = decode_predictions(scores, geometry)
        boxes = non_max_suppression(np.array(rects), probs=confidences)
 
    # loop over the bounding boxes
        for (startX, startY, endX, endY) in boxes:
        # scale the bounding box coordinates based on the respective
        # ratios
            startX = int(startX * rW)
            startY = int(startY * rH)
            endX = int(endX * rW)
            endY = int(endY * rH)
 
        # draw the bounding box on the frame
            cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)
# update the FPS counter
        fps.update()
 
    # show the output frame
        cv2.imshow("Text Detection", orig)
        key = cv2.waitKey(1) & 0xFF
 
    # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
 
 # stop the timer and display FPS information
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
 
 # orelease the file pointer
    vs.release()
 
 # close all windows
    cv2.destroyAllWindows()


eel.start('index.html')
